In [53]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
import pandas as pd
import gradio as gr

load_dotenv(override=True)

True

In [54]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [55]:
system_message = "You are a helpful assistant for recommending products from an insurance company. "
system_message += "Give a short, friendly answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so and mention the reason provided to you."
system_message += "If you have found products, describe them with their name, product_id and price."

In [56]:
n_products = 100

# Description templates
descriptions = {
    "car": {
        "full": [
            "Comprehensive car coverage including accidents and theft.",
            "All-round protection for your vehicle.",
            "Covers collisions, liability, and more."
        ],
        "partial": [
            "Basic car insurance for liability coverage.",
            "Covers third-party damages only.",
            "Affordable plan for essential coverage."
        ]
    },
    "healthcare": {
        "full": [
            "Full healthcare plan covering hospital visits and medication.",
            "Complete medical protection with dental and vision included.",
            "Premium healthcare for individuals and families."
        ],
        "partial": [
            "Basic medical coverage for general checkups.",
            "Affordable plan covering essential health needs.",
            "Partial plan for outpatient services."
        ]
    },
    "property": {
        "full": [
            "Protects your home against fire, theft, and natural disasters.",
            "Comprehensive property insurance with contents cover.",
            "All-in-one protection for buildings and belongings."
        ],
        "partial": [
            "Covers basic structural damages.",
            "Entry-level property insurance for homeowners.",
            "Partial protection plan for small incidents."
        ]
    }
}

# Generate products
df_data = []

for i in range(10):
    price = np.random.randint(100, 200)
    category = np.random.choice(["car", "healthcare", "property"])
    insurance_type = "full" if price > 150 else "partial"
    description = random.choice(descriptions[category][insurance_type])

    # Generate a fake product URL
    product_url = f"https://www.exampleinsurance.com/products/{category}/{i}"

    df_data.append({
        "product_id": i,
        "product_name": f"Product {i}",
        "price": price,
        "category": category,
        "type": insurance_type,
        "description": description,
        "product_url": product_url
    })

# Convert to DataFrame
insurance_products_df = pd.DataFrame(df_data)

print(insurance_products_df.head())

   product_id product_name  price    category     type  \
0           0    Product 0    162    property     full   
1           1    Product 1    183         car     full   
2           2    Product 2    102         car  partial   
3           3    Product 3    163  healthcare     full   
4           4    Product 4    163  healthcare     full   

                                         description  \
0  All-in-one protection for buildings and belong...   
1            Covers collisions, liability, and more.   
2                   Covers third-party damages only.   
3  Complete medical protection with dental and vi...   
4   Premium healthcare for individuals and families.   

                                         product_url  
0  https://www.exampleinsurance.com/products/prop...  
1    https://www.exampleinsurance.com/products/car/1  
2    https://www.exampleinsurance.com/products/car/2  
3  https://www.exampleinsurance.com/products/heal...  
4  https://www.exampleinsurance.com/pro

In [57]:
def get_recommendation(df: pd.DataFrame, category: str, price_limit: float = np.infty, desired_type: str = "full") -> pd.DataFrame:
    # Filter by category
    filtered_df = df[df["category"] == category]

    # Further filter by price
    filtered_df = filtered_df[filtered_df["price"] <= price_limit]

    # Further filter by desired_type
    filtered_df = filtered_df[filtered_df["type"] == desired_type]

    return filtered_df

In [61]:
recommendation_tool = {
    "name": "get_recommendation",
    "description": "Get the recommendations for the insurance products. Call this function whenever the user asks for a product recommendation, such as 'I'm looking for a partial healthcare insurance that covers my basic needs and costs no more than $130. The function returns the entries in the database that fulfill the given criteria. If there are no entries that fulfull the criteria, give recommendations how to resolve this. E.g., you may respond with 'Try increasing your price limit'. The result are the contents of the dataframe as a dictionary. If you have already given all available information to the user, say so.",
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "description": "The category of the insurance product. Can either be 'car', 'healthcare', or 'property'."
            },
            "price_limit": {
                "type": "number",
                "description": "The maximum price of the insurance product."
            },
            "desired_type": {
                "type": "string",
                "description": "The type of coverage the insurance provides. Either 'full' or 'partial'."
            }
        },
        "required": ["category"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": recommendation_tool}]

In [62]:
def call_tool(message, df):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    category = arguments.get('category', "healthcare")
    price_limit = arguments.get('price_limit', np.infty)
    desired_type = arguments.get('desired_type', "full")
    product_recommendation = get_recommendation(df, category, price_limit, desired_type)
    response = {
        "role": "tool",
        "content": json.dumps(product_recommendation.to_dict()),
        "tool_call_id": tool_call.id
    }
    return response, product_recommendation

def chat(message, history, df, tools, model):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, _ = call_tool(message, df)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=model, messages=messages)

    return response.choices[0].message.content

In [63]:
gr.ChatInterface(fn=lambda m, h: chat(m, h, insurance_products_df, tools, MODEL),
                 type="messages").launch(show_error=True)

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
